##QF Fall Project 1

Diana Alvarez, Daniel Ricci, Mike Savino

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np

##PART 1: Create a table showing constituent (stocks) risk analysis in the equal-weight portfolio analysis as of the current date

##Column 1

In [ ]:
assets = ["GOOG", "AMZN", "MSFT", "NVDA", "META", "TSLA", "AAPL"]

##Column 2

Portfolio Weight for each asset = 1/number of asset --> 7 stocks, 1/7 weight

In [ ]:
weights = np.array([1 / len(assets)] * len(assets))
print(weights)

[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]


##Column 3

Annualized Volatility: daily volatility x Root252

In [ ]:
end_date = datetime.today()
start_date = end_date - timedelta(days=90)

prices_3months =yf.download(assets, start=start_date, end=end_date)['Adj Close']
returns = prices_3months.pct_change()
daily_volatility = returns.std()

annualized_volatility = daily_volatility * np.sqrt(252)
volatility_3mo= np.sqrt(np.dot(weights.T, np.dot(np.diag(annualized_volatility**2), weights)))


[*********************100%***********************]  7 of 7 completed


In [ ]:
print(volatility_3mo)

0.15697183788988822


##COLUMN 4,5,6
Beta against SPY: A beta coefficient shows the volatility of an individual stock compared to the systematic risk of the entire market.

Beta coefficient(β)= Covariance(Re,Rm)/Variance(Rm)

re, return of indivdual stock
rm, retun on the overall market
covariance, changes in stock's return relation to change's in the market's return
variance, the spread of market data points compared to the average

sources: https://www.investopedia.com/terms/b/beta.asp#:~:text=cost%20of%20capital.-,Calculating%20Beta,the%20rest%20of%20the%20market
https://news.ycombinator.com/item?id=3142098

In [ ]:
benchmarks= ["SPY", "IWM", "DIA"]
beta_end_date= datetime.now()
beta_start_date = end_date - timedelta(days=356)

benchmark_prices = yf.download(assets + benchmarks, start=beta_start_date, end=beta_end_date)['Adj Close']
benchmark_return = benchmark_prices.pct_change()
benchmark_returns = benchmark_return.dropna()
print(benchmark_returns)

[*********************100%***********************]  10 of 10 completed

Ticker                         AAPL      AMZN       DIA      GOOG       IWM  \
Date                                                                          
2023-10-11 00:00:00+00:00  0.007904  0.018150  0.001719  0.017960 -0.001705   
2023-10-12 00:00:00+00:00  0.005061  0.003793 -0.005207 -0.009951 -0.022599   
2023-10-13 00:00:00+00:00 -0.010293 -0.019195  0.001219 -0.012189 -0.008329   
2023-10-16 00:00:00+00:00 -0.000727  0.021265  0.009534  0.013783  0.016151   
2023-10-17 00:00:00+00:00 -0.008785 -0.008148  0.000294  0.003559  0.011155   
...                             ...       ...       ...       ...       ...   
2024-09-24 00:00:00+00:00  0.003974  0.000413  0.002422  0.003495  0.001676   
2024-09-25 00:00:00+00:00 -0.004398 -0.007373 -0.007131 -0.003972 -0.012197   
2024-09-26 00:00:00+00:00  0.005080 -0.007116  0.005965  0.005154  0.005329   
2024-09-27 00:00:00+00:00  0.001187 -0.016688  0.003155  0.008912  0.006763   
2024-09-30 00:00:00+00:00  0.015067 -0.012662 -0.003

beta= covar of asset and becnhmark returns/var of benchmark returns
source: https://www.investopedia.com/terms/b/beta.asp#:~:text=cost%20of%20capital.-,Calculating%20Beta,the%20rest%20of%20the%20market.

In [ ]:
def calc_beta(asset_returns, benchmark_returns):
    covariance = asset_returns.cov(benchmark_returns)
    variance = benchmark_returns.var()
    beta = covariance / variance
    return beta

beta_against_IWN = {}
for asset in assets:
    stock_returns = returns[asset]
    beta_against_IWN[asset] = calc_beta(stock_returns, benchmark_returns['IWM'])

beta_against_DIA = {}
for asset in assets:
    stock_returns = returns[asset]
    beta_against_DIA[asset] = calc_beta(stock_returns, benchmark_returns['DIA'])

beta_against_SPY = {}
for asset in assets:
    stock_returns = returns[asset]
    beta_against_SPY[asset] = calc_beta(stock_returns, benchmark_returns['SPY'])


In [ ]:
print(beta_against_SPY)
print(beta_against_IWN)
print(beta_against_DIA)

{'GOOG': 1.2840358123061855, 'AMZN': 1.4934279132118717, 'MSFT': 1.0790059920957562, 'NVDA': 2.733391386907724, 'META': 1.5853641633863287, 'TSLA': 2.2235020891826864, 'AAPL': 0.9998123361699232}
{'GOOG': 0.37421345415060725, 'AMZN': 0.4840499056753191, 'MSFT': 0.24971658894797666, 'NVDA': 0.638424224576805, 'META': 0.37663006791635006, 'TSLA': 1.025977962123993, 'AAPL': 0.3298066955393905}
{'GOOG': 0.8872812249916401, 'AMZN': 1.1640888280460224, 'MSFT': 0.9047999537641851, 'NVDA': 1.2584601018244959, 'META': 1.0505318232264473, 'TSLA': 1.8154001237248296, 'AAPL': 0.8003109289437168}


##Column 7, 8
Average weekly drawdown & Max weekly drawdown  (52 week LOW- 52 week HIGH)/52-week high

initial attempt--> errors

In [ ]:
awd_end_date= datetime.now()
awd_start_date = end_date - timedelta(weeks=52)

awd_prices = yf.download(assets, start=awd_start_date, end=awd_end_date)['Adj Close']
import pandas as pd

drawdown = {}
for asset in assets:
    weekly_high = awd_prices[asset].max()
    weekly_low = awd_prices[asset].min()
    awd= weekly_low - weekly_high
    drawdown[asset] = awd / weekly_high

drawdown.mean()
drawdown.min()

[*********************100%***********************]  7 of 7 completed


AttributeError: 'dict' object has no attribute 'mean'

Modifications after CHATGPT

In [ ]:
average_weekly_drawdown = {}
maximum_weekly_drawdown = {}


for asset in assets:
    weekly_prices = awd_prices[asset].resample('W').agg(['max', 'min'])
    drawdowns = (weekly_prices['min'] - weekly_prices['max']) / weekly_prices['max']
    average_weekly_drawdown[asset] = drawdowns.mean()
    maximum_weekly_drawdown[asset] = drawdowns.min()

print (average_weekly_drawdown)
print (maximum_weekly_drawdown)

{'GOOG': -0.03313184859337624, 'AMZN': -0.033948386666553476, 'MSFT': -0.025322075956974166, 'NVDA': -0.061957249990517424, 'META': -0.04202629858492829, 'TSLA': -0.06431517820634379, 'AAPL': -0.02690647043017974}
{'GOOG': -0.1193262898333786, 'AMZN': -0.10204301137570579, 'MSFT': -0.05945829521067111, 'NVDA': -0.14394095552251854, 'META': -0.17863524028934608, 'TSLA': -0.1681773575981104, 'AAPL': -0.09858109553486116}


##Column 9

total return (using trailing 10 years=520 weeks) : ((o-n)-1)*100

source: https://stackoverflow.com/questions/63434019/

-total-returns-from-a-dataframe

((price 10 years ago/current price )-1)*100 = total return

In [ ]:
total_10_end_date= datetime.now()
total_10_start_date = end_date - timedelta(weeks=520)

total_10_prices = yf.download(assets, start=total_10_start_date, end=total_10_end_date)['Adj Close']

total_returns = {}
for asset in assets:
    beginning_value = total_10_prices[asset].iloc[0]
    ending_value = total_10_prices[asset].iloc[-1]

    total_return = ((ending_value / beginning_value) - 1) * 100
    total_returns[asset] = total_return

print(total_returns)

[*********************100%***********************]  7 of 7 completed

{'GOOG': 525.7547880817182, 'AMZN': 1110.9642700608945, 'MSFT': 1044.2975664451742, 'NVDA': 29842.012448371595, 'META': 682.277041634677, 'TSLA': 1628.8169875896012, 'AAPL': 943.5781751940126}


##Column 10
Annualized Total Return

In [ ]:
annual_end_date= datetime.now()
annual_start_date = end_date - timedelta(weeks=520)

annualized_total_prices = yf.download(assets, start=annual_start_date, end=annual_end_date)['Adj Close']
annualized_returns = annualized_total_prices.pct_change().dropna()

annualized_total_returns = {}
for asset in assets:
    annualized_return = ((1 + total_return) ** (1 / 10)) - 1
    total_returns[asset] = total_return * 100
    annualized_total_returns[asset] = annualized_return * 100


print(annualized_total_returns)

[*********************100%***********************]  7 of 7 completed

{'GOOG': 98.39183330499924, 'AMZN': 98.39183330499924, 'MSFT': 98.39183330499924, 'NVDA': 98.39183330499924, 'META': 98.39183330499924, 'TSLA': 98.39183330499924, 'AAPL': 98.39183330499924}


##TABLE 1

In [ ]:
portfolio_dict= ({'Ticker': assets,
       'Portfolio Weight': weights,
       'Annualized Volatility': annualized_volatility,
            'Beta against SPY': beta_against_SPY,
            'Beta against IWM': beta_against_IWN,
            'Beta against DIA': beta_against_DIA,
            'Average Weekly Drawdown': average_weekly_drawdown,
            'Max Weekly Drawdown': maximum_weekly_drawdown,
            'Total Return': total_returns,
            'Annualized Total Return': annualized_total_returns
            })

print(portfolio_dict)

{'Ticker': ['GOOG', 'AMZN', 'MSFT', 'NVDA', 'META', 'TSLA', 'AAPL'], 'Portfolio Weight': array([0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
       0.14285714, 0.14285714]), 'Annualized Volatility': Ticker
AAPL    0.225577
AMZN    0.324024
GOOG    0.264659
META    0.347259
MSFT    0.201909
NVDA    0.653122
TSLA    0.627314
dtype: float64, 'Beta against SPY': {'GOOG': 2.017826405767726, 'AMZN': 2.512683017624418, 'MSFT': 1.6095222840895576, 'NVDA': 5.340173285120893, 'META': 2.148026468116663, 'TSLA': 4.71236754043569, 'AAPL': 1.5686673522645533}, 'Beta against IWM': {'GOOG': 0.6774154066393676, 'AMZN': 0.8986675207138117, 'MSFT': 0.44718717547083764, 'NVDA': 1.5041546870591034, 'META': 0.35013745956006115, 'TSLA': 1.5184881542657205, 'AAPL': 0.520493140592844}, 'Beta against DIA': {'GOOG': 1.6347432747485828, 'AMZN': 2.1685899291941206, 'MSFT': 1.3303787964172107, 'NVDA': 3.420345364557264, 'META': 1.2159255636339308, 'TSLA': 3.572596817994636, 'AAPL': 1.339244272327688}

Initial attempt to create dataframe:

In [ ]:
# @title
#columns = ['Ticker', 'Portfolio Weight', 'Annualized Volatility', 'Beta against SPY', 'Beta against IWM', 'Beta against DIA', 'Average Weekly Drawdown', 'Max Weekly Drawdown', 'Total Return', 'Annualized Total Return']
#portfolio_df = pd.DataFrame(portfolio)
#portfolio_df = portfolio_df[columns]

#ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

Attempts to reconfig code with assistance of ChatGPT
By calling list() on each value, especially dictionary .values(), you standardize the format and avoid scalar values causing the issue.

list(beta_against_SPY.values())



In [ ]:
menu = {
    "dish": ["penne alla vodka", "panna cotta", "spaghetti arrabbiata", "caprese", "MVP", "pizza", "pesto gnocchi", "tiramisu", "gelato", "bruschetta", "eggplant parm"],
    "category": ["pasta", "dessert", "pasta", "salad", "pizza", "pizza", "pasta", "dessert","dessert", "appetizers", "mains"],
    "price": [11,14,15,17,18,19,23,25,26,27,28]
}

menu_df = pd.DataFrame(menu)

menu_df.head()

,dish,category,price
0,penne alla vodka,pasta,11
1,panna cotta,dessert,14
2,spaghetti arrabbiata,pasta,15
3,caprese,salad,17
4,MVP,pizza,18


In [ ]:
import pandas as pd
portfolio = {
    'Ticker': assets,
    'Portfolio Weight': weights,
    'Annualized Volatility': list(annualized_volatility),
    'Beta against SPY': list(beta_against_SPY.values()),
    'Beta against IWM': list(beta_against_IWN.values()),
    'Beta against DIA': list(beta_against_DIA.values()),
    'Average Weekly Drawdown': list(average_weekly_drawdown.values()),
    'Max Weekly Drawdown': list(maximum_weekly_drawdown.values()),
    'Total Return': list(total_returns.values()),
    'Annualized Total Return': list(annualized_total_returns.values())
}

portfolio_df = pd.DataFrame(portfolio)
portfolio_df.set_index('Ticker', inplace=True)

columns = ['Portfolio Weight', 'Annualized Volatility', 'Beta against SPY', 'Beta against IWM', 'Beta against DIA', 'Average Weekly Drawdown', 'Max Weekly Drawdown', 'Total Return', 'Annualized Total Return']

portfolio_df = portfolio_df[columns]

print(portfolio_df)


        Portfolio Weight  Annualized Volatility  Beta against SPY  \
Ticker                                                              
GOOG            0.142857               0.225577          2.017826   
AMZN            0.142857               0.324024          2.512683   
MSFT            0.142857               0.264659          1.609522   
NVDA            0.142857               0.347259          5.340173   
META            0.142857               0.201909          2.148026   
TSLA            0.142857               0.653122          4.712368   
AAPL            0.142857               0.627314          1.568667   

        Beta against IWM  Beta against DIA  Average Weekly Drawdown  \
Ticker                                                                
GOOG            0.677415          1.634743                -0.033132   
AMZN            0.898668          2.168590                -0.033948   
MSFT            0.447187          1.330379                -0.025322   
NVDA            1.50415

##Part 2: Portfolio Risk Against the Three ETFs

##Column 1

In [ ]:
import pandas as pd
assets = ["GOOG", "AMZN", "MSFT", "NVDA", "META", "TSLA", "AAPL"]
benchmarks = ["SPY", "IWM", "DIA"]

##Column 2, 3

import numpy as np

x represents the total sale in dollars
x = [215, 325, 185, 332, 406, 522, 412,
     614, 544, 421, 445, 408],

y represents the temperature on each day of sale
y = [14.2, 16.4, 11.9, 15.2, 18.5, 22.1,
     19.4, 25.1, 23.4, 18.1, 22.6, 17.2]

create correlation matrix
matrix = np.corrcoef(x, y)
print(matrix)

source: Geeksforgeeks.org

DataFrame.corr(method='pearson', min_periods=1, numeric_only=False)

Guiding code: Pandas.pydata.org and G

In [ ]:
corr_prices = yf.download(assets + benchmarks, start=start_date, end=end_date)['Adj Close']
corr_returns = corr_prices.pct_change().dropna()


corr_end_date = datetime.today()
corr_start_date = end_date - timedelta(days=365)

correlation_matrix = corr_returns.corr()
covariance_matrix = corr_returns.cov()


correlation_with_benchmarks = correlation_matrix.loc[assets, benchmarks]
covariance_with_benchmarks = covariance_matrix.loc[assets, benchmarks]

print(correlation_with_benchmarks)
print(covariance_with_benchmarks)

[*********************100%***********************]  10 of 10 completed


Ticker       SPY       IWM       DIA
Ticker                              
GOOG    0.743602  0.479198  0.544303
AMZN    0.756395  0.519478  0.589637
MSFT    0.777557  0.414756  0.580567
NVDA    0.797549  0.431354  0.461543
META    0.603415  0.188852  0.308713
TSLA    0.732748  0.453367  0.501894
AAPL    0.678462  0.432052  0.523751
Ticker       SPY       IWM       DIA
Ticker                              
GOOG    0.000124  0.000125  0.000075
AMZN    0.000155  0.000166  0.000099
MSFT    0.000099  0.000082  0.000061
NVDA    0.000329  0.000278  0.000156
META    0.000132  0.000065  0.000056
TSLA    0.000290  0.000280  0.000163
AAPL    0.000097  0.000096  0.000061


##Column  4

Tracking error: standard deviation for the difference between the portfolio returns and the benchmark (ETF) returns.

In [ ]:

tracking_end_date = datetime.today()
tracking_start_date = tracking_end_date - timedelta(days=3650)
trackingerror_prices = yf.download(benchmarks, start=tracking_start_date, end=tracking_end_date)['Adj Close']

trackingerror_returns = trackingerror_prices.pct_change().dropna()

tracking_errorspy = (trackingerror_returns - trackingerror_returns['SPY']).abs().mean()
tracking_erroriwn = (trackingerror_returns - trackingerror_returns['IWM']).abs().mean()
tracking_errordia = (trackingerror_returns - trackingerror_returns['DIA']).abs().mean()


[*********************100%***********************]  3 of 3 completed


,0
2024-09-20 00:00:00+00:00,NaN
2024-09-23 00:00:00+00:00,NaN
2024-09-24 00:00:00+00:00,NaN
2024-09-25 00:00:00+00:00,NaN
2024-09-26 00:00:00+00:00,NaN
2024-09-27 00:00:00+00:00,NaN
2024-09-30 00:00:00+00:00,NaN
DIA,NaN
IWM,NaN
SPY,NaN


##Column 5: Sharp Ratio (using risk free rate)

sharpe ratio= (portfolio return - risk free rate)/ portfolio volatility

Return from a risk-free investment (e.g., U.S. Treasury bond yield). You can use a constant value like 0.02 (or 2%) as a placeholder.

In [ ]:
# @title
sr_end_date = datetime.today()
sr_start_date = sr_end_date - timedelta(days=365)

sr_prices = yf.download(assets + benchmarks, start=sr_start_date, end=sr_end_date)['Adj Close']
sr_returns = sr_prices.pct_change().dropna()
sr_mean_returns = sr_returns.mean()

sharpe_ratio = sr_returns.mean() / sr_returns.std()
print(sharpe_ratio)


[*********************100%***********************]  10 of 10 completed

Ticker
AAPL    0.090409
AMZN    0.090954
DIA     0.151086
GOOG    0.055933
IWM     0.079687
META    0.119031
MSFT    0.100054
NVDA    0.137197
SPY     0.158267
TSLA    0.020636
dtype: float64


##Column 6
Volatility spread  =  portfolio-benchmark

In [ ]:
sp_end_date = datetime.today()
sp_start_date = sp_end_date - timedelta(days=365)

sp_prices = yf.download(assets, start=sp_start_date, end=sp_end_date)['Adj Close']
sp_returns = sp_prices.pct_change()
sp_daily_volatility = sp_returns.std()
sp_annualized_volatility = sp_daily_volatility * np.sqrt(252)

portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(np.diag(sp_annualized_volatility**2), weights)))

print(portfolio_volatility)

[*********************100%***********************]  7 of 7 completed

0.1392050540536427


In [ ]:
bweights = np.array([1 / len(benchmarks)] * len(benchmarks))
print(bweights)

spb_prices = yf.download(benchmarks, start=sp_start_date, end=sp_end_date)['Adj Close']
spb_returns = spb_prices.pct_change()
spb_daily_volatility = spb_returns.std()
spb_annualized_volatility = spb_daily_volatility * np.sqrt(252)

benchmark_volatility = np.sqrt(np.dot(bweights.T, np.dot(np.diag(spb_annualized_volatility**2), bweights)))

print(benchmark_volatility)

[                       0%                       ]

[0.33333333 0.33333333 0.33333333]


[*********************100%***********************]  3 of 3 completed

0.09001743538971012


In [ ]:
volatility_spread= portfolio_volatility - benchmark_volatility
print(volatility_spread)

0.04918761866393258


##Table 2

Same procress as before, Correlation against ETF': list(correlation_with_benchmarks.values()) and so on....

In [ ]:

benchmark_dict= {
    'ETFs': benchmarks,
    'Correlation against ETF': correlation_with_benchmarks,
    'Covariance of Portfolio against ETF': covariance_with_benchmarks,
    'Tracking Error': tracking_errors,
    'Sharpe Ratio': sharpe_ratio,
    'Volatility Spread': volatility_spread
}

print(benchmark_dict)


{'ETFs': ['SPY', 'IWM', 'DIA'], 'Correlation against ETF': Ticker       SPY       IWM       DIA
Ticker                              
GOOG    0.744087  0.479673  0.544600
AMZN    0.756926  0.519638  0.589202
MSFT    0.777369  0.414532  0.579488
NVDA    0.797585  0.431287  0.461120
META    0.603107  0.188590  0.307634
TSLA    0.732732  0.453244  0.501185
AAPL    0.676487  0.430710  0.519814, 'Covariance of Portfolio against ETF': Ticker       SPY       IWM       DIA
Ticker                              
GOOG    0.000124  0.000125  0.000075
AMZN    0.000155  0.000166  0.000099
MSFT    0.000099  0.000082  0.000061
NVDA    0.000329  0.000277  0.000156
META    0.000132  0.000065  0.000055
TSLA    0.000290  0.000280  0.000163
AAPL    0.000096  0.000096  0.000061, 'Tracking Error': {('GOOG', 'SPY'): 0.012505239871177402, ('GOOG', 'IWM'): 0.015366113798060016, ('GOOG', 'DIA'): 0.014007910663515932, ('AMZN', 'SPY'): 0.016270880520859145, ('AMZN', 'IWM'): 0.018671115252555364, ('AMZN', 'DIA'): 0.0

##PART 3: Correlation Matrix

In [ ]:

corrm_end_date = datetime.today()
corrm_start_date = end_date - timedelta(days=365)
corrm_prices = yf.download(assets+benchmarks, start=corrm_start_date, end=corrm_end_date)['Adj Close']
corrm_returns = corrm_prices.pct_change().dropna()


correlation_matrix = corrm_returns.corr()

print(correlation_matrix)


[*********************100%***********************]  10 of 10 completed

Ticker      AAPL      AMZN       DIA      GOOG       IWM      META      MSFT  \
Ticker                                                                         
AAPL    1.000000  0.379818  0.389424  0.406868  0.321672  0.248179  0.471147   
AMZN    0.379818  1.000000  0.456712  0.554270  0.380209  0.611291  0.617431   
DIA     0.389424  0.456712  1.000000  0.347221  0.758787  0.315381  0.511051   
GOOG    0.406868  0.554270  0.347221  1.000000  0.292860  0.416983  0.529134   
IWM     0.321672  0.380209  0.758787  0.292860  1.000000  0.226832  0.285718   
META    0.248179  0.611291  0.315381  0.416983  0.226832  1.000000  0.564039   
MSFT    0.471147  0.617431  0.511051  0.529134  0.285718  0.564039  1.000000   
NVDA    0.311402  0.483074  0.264254  0.374846  0.266871  0.417169  0.468158   
SPY     0.559418  0.671630  0.828292  0.577501  0.727747  0.542413  0.696833   
TSLA    0.396073  0.302401  0.354978  0.222449  0.400342  0.174437  0.275715   

Ticker      NVDA       SPY      TSLA  
